In [113]:
import pandas as pd 
import plotly.express as px
import plotly.graph_objects as go

Cel: Profilowanie społeczno-demograficzne migrantów
* Zbudowanie profili migrantów (wiek, płeć, wykształcenie, status zawodowy) migrujących do krajów UE, jak i do poszczególnych krajów spośród tych wybrancyh do analizy - liberal = ["France", "Italy", "Germany", "Sweden", "Spain", "Austria"], conservs = ["Poland", "Bulgaria", "Romania", "Hungary",  "Greece"]
* kierunki i wzorce dotyczące mirgacji są już dostępne w migration_patterns - w tej częsci całkowicie to pominięto, zapobiegając redundancji
* wybrane zbiory danych:
    * Population on 1 January by age group, sex and citizenship
    * Population by educational attainment level, sex, age and citizenship (%)
    * Population by sex, age, citizenship and labour status (1 000)
    (jak wszystkie dane dotyczące zatrudnień - te również - obejmują osoby (15-64))

# Population on 1 January by age group, sex and citizenship

In [78]:
df = pd.read_csv('../processed_data/population_foreigners.csv')

In [79]:
df.head()

,Sex,geo,Age class,year,Country of citizenship,Number
0,Females,Austria,Total,2012,Foreign country and stateless,470822
1,Females,Austria,Total,2013,Foreign country and stateless,496844
2,Females,Austria,Total,2014,Foreign country and stateless,526542
3,Females,Austria,Total,2015,Foreign country and stateless,562672
4,Females,Austria,Total,2016,Foreign country and stateless,611580


In [80]:
liberal = ["France", "Italy", "Germany", "Sweden", "Spain", "Austria"]
conservs = ["Poland", "Bulgaria", "Romania", "Hungary",  "Greece"]
selected = liberal + conservs
df['Group'] = df['geo'].apply(
    lambda x: 'Liberal' if x in liberal else ('Conservative' if x in conservs else 'Other')
)
only_foreigners = df[df['Country of citizenship'] == 'Foreign country and stateless']

### Struktura wiekowa migrantów wg płci i kraju

In [81]:
age_order = [
    'Less than 15 years', 'From 15 to 19 years', 'From 20 to 24 years',
    'From 25 to 29 years', 'From 30 to 34 years', 'From 35 to 39 years',
    'From 40 to 44 years', 'From 45 to 49 years', 'From 50 to 54 years',
    'From 55 to 59 years', 'From 60 to 64 years', '65 years or over'
]

In [82]:
def plot_age_structure(df, title, age_order):
    age_structure = df.groupby(['geo', 'Sex', 'Age class'], as_index=False)['Number'].mean()
    age_structure = age_structure[age_structure['Age class'].isin(age_order)]
    age_structure['Age class'] = pd.Categorical(age_structure['Age class'], categories=age_order, ordered=True)
    fig = px.bar(
    age_structure,
    x='Age class',
    y='Number',
    color='Sex',
    facet_col='geo',
    facet_col_wrap=3,
    category_orders={'Age class': age_order},
    title=title,
    labels={'Age class': 'Age class', 'Number': 'Foreigners number'}
)

    fig.update_layout(height=1000, width=1200, showlegend=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.show()

In [83]:
plot_age_structure(only_foreigners[(only_foreigners['geo'].isin(liberal))
                                    & (only_foreigners['Sex'] != 'Total')
                                    & (only_foreigners['year'] >= 2014)],
                   'Age structure of foreigners in selected liberal countries', age_order)

In [84]:
plot_age_structure(only_foreigners[(only_foreigners['geo'].isin(conservs))
                                    & (only_foreigners['Sex'] != 'Total') &
                                    (only_foreigners['year'] >= 2014)],
                   'Age structure of foreigners in selected conservative countries', age_order)

In [93]:
age_merge = {
    'Less than 15 years': '0-14',
    'From 15 to 19 years': '15-24',
    'From 20 to 24 years': '15-24',
    'From 25 to 29 years': '25-34',
    'From 30 to 34 years': '25-34',
    'From 35 to 39 years': '35-49',
    'From 40 to 44 years': '35-49',
    'From 45 to 49 years': '35-49',
    'From 50 to 54 years': '50-64',
    'From 55 to 59 years': '50-64',
    'From 60 to 64 years': '50-64',
    '65 years or over': '65+'
}
new_age_order = ['0-14', '15-24', '25-34', '35-49', '50-64', '65+']

temp_df = only_foreigners.copy()

temp_df['Age class'] = temp_df['Age class'].map(age_merge)

plot_age_structure(temp_df[(temp_df['geo'].isin(liberal))
                            & (temp_df['Sex'] != 'Total')
                            & (temp_df['year'] >= 2014)],
                   'Age structure of foreigners in selected liberal countries',
                   new_age_order)


plot_age_structure(temp_df[(temp_df['geo'].isin(conservs)) &
                            (temp_df['Sex'] != 'Total') &
                            (only_foreigners['year'] >= 2014)],
                   'Age structure of foreigners in selected conservative countries',
                   new_age_order)

Wszystkie analizy dla ostatnich 10 lat (2014-2024). Jako średnia liczba na przestrzeni ostatnich lat.

In [86]:
# 2024 - latest data
latest_df = temp_df[temp_df['year'] == 2024]
plot_age_structure(latest_df[(latest_df['geo'].isin(liberal))
                            & (latest_df['Sex'] != 'Total')],
                   'Age structure of foreigners in selected liberal countries in 2024',
                   new_age_order)


plot_age_structure(latest_df[(latest_df['geo'].isin(conservs)) &
                            (latest_df['Sex'] != 'Total')],
                   'Age structure of foreigners in selected conservative countries in 2024',
                   new_age_order)

In [95]:
fig = px.line(
    temp_df[(temp_df['geo'].isin(liberal)) & (temp_df['Sex'] == 'Total')].dropna().groupby(['geo', 'year', 'Age class'], as_index=False)['Number'].sum(),
    x="year",
    y="Number",
    color="Age class",
    facet_col="geo",
    facet_col_wrap=3,
    markers=True,
    title="Trends in education levels of foreigners in selected liberal countries (2014-2024)",
    labels={
        "year": "Year",
        "Number": "Number of Foreigners in Population",
        "Age class": "Age class"
    }
)
fig.update_layout(height=800, width=1200)
fig.show()


In [96]:
fig = px.line(
    temp_df[(temp_df['geo'].isin(conservs)) & (temp_df['Sex'] == 'Total')].dropna().groupby(['geo', 'year', 'Age class'], as_index=False)['Number'].sum(),
    x="year",
    y="Number",
    color="Age class",
    facet_col="geo",
    facet_col_wrap=3,
    markers=True,
    title="Trends in education levels of foreigners in selected convervative countries (2014-2024)",
    labels={
        "year": "Year",
        "Number": "Number of Foreigners in Population",
        "Age class": "Age class"
    }
)
fig.update_layout(height=800, width=1200)
fig.show()

### Struktura płciowa migrantów 

In [54]:
import plotly.express as px

def plot_sex_distribution(df, title):
    sex_structure = df.groupby(['geo', 'Sex'], as_index=False)['Number'].mean()
    
    fig = px.bar(
        sex_structure,
        x='Sex',
        y='Number',
        color='Sex',
        facet_col='geo',
        facet_col_wrap=3,
        title=title,
        labels={'Sex': 'Sex', 'Number': 'Number of foreigners'}
    )

    fig.update_layout(
        height=1000,
        width=1200,
        showlegend=False
    )
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.show()

In [56]:
only_foreigners_liberal_f = only_foreigners[
    (only_foreigners['geo'].isin(liberal)) &  
    (only_foreigners['Sex'].isin(['Males', 'Females'])) &
    (only_foreigners['Age class'] == 'Total') &
    (only_foreigners['year'] >= 2014)
]

only_foreigners_conservs_f = only_foreigners[
    (only_foreigners['geo'].isin(conservs)) &  
    (only_foreigners['Sex'].isin(['Males', 'Females'])) &
    (only_foreigners['Age class'] == 'Total') &
    (only_foreigners['year'] >= 2014)
]

plot_sex_distribution(only_foreigners_liberal_f, title='Structure of foreigners in selected liberal countries (2014-2024, mean annual)')
plot_sex_distribution(only_foreigners_conservs_f, title='Structure of foreigners in selected conservative countries (2014-2024, mean annual)')



50 na 50 dla wszystkich analiz

# Population by educational attainment level, sex, age and citizenship (%)

In [98]:
df = pd.read_csv('../processed_data/education_levels_foreigners.csv')

In [99]:
df.head()

,ISCED,geo,year,Percentage
0,levels 0-2,Austria,2014,29.6
1,levels 0-2,Austria,2015,28.1
2,levels 0-2,Austria,2016,28.6
3,levels 0-2,Austria,2017,28.9
4,levels 0-2,Austria,2018,28.6


In [100]:
fig = px.line(
    df[df['geo'].isin(liberal)],
    x="year",
    y="Percentage",
    color="ISCED",
    facet_col="geo",
    facet_col_wrap=3,
    markers=True,
    title="Trends in education levels of foreigners in selected liberal countries (2014-2024)",
    labels={
        "year": "Year",
        "Percentage": "Percentage of foreigners",
        "ISCED": "ISCED Level"
    }
)
fig.update_layout(height=800, width=1200)
fig.show()


In [101]:
fig = px.line(
    df[df['geo'].isin(conservs)],
    x="year",
    y="Percentage",
    color="ISCED",
    facet_col="geo",
    facet_col_wrap=3,
    markers=True,
    title="Trends in education levels of foreigners in selected conservative countries (2014-2024)",
    labels={
        "year": "Year",
        "Percentage": "Percentage of foreigners",
        "ISCED": "ISCED Level"
    }
)
fig.update_layout(height=800, width=1200)
fig.show()


In [102]:
last_year = df['year'].max()
last_year_df = df[df['year'] == last_year]

fig = px.bar(
    last_year_df,
    x='geo',
    y='Percentage',
    color='ISCED',
    barmode='group',
    title=f"Percentage of educational levels among foreigners in {last_year}",
    labels={'geo': 'Country', 'Percentage': 'foreigners (%)', 'ISCED': 'ISCED'}
)
fig.update_layout(height=500, width=1000)
fig.show()


In [103]:
last_year = df['year'].max()
last_year_df = df[df['year'] == last_year]

fig = px.bar(
    last_year_df[last_year_df['geo'].isin(selected)],
    x='geo',
    y='Percentage',
    color='ISCED',
    barmode='group',
    title=f"Percentage of educational levels among foreigners in {last_year} in selected countries",
    labels={'geo': 'Country', 'Percentage': 'foreigners (%)', 'ISCED': 'ISCED'}
)
fig.update_layout(height=500, width=1000)
fig.show()

In [104]:
fig = px.line(
    df[df['geo'].isin(liberal)],
    x="year",
    y="Percentage",
    color="ISCED",
    facet_col="geo",
    facet_col_wrap=3,
    markers=True,
    title="Trends in education levels of foreigners in selected liberal countries (2014-2024)",
    labels={
        "year": "Year",
        "Percentage": "Percentage of foreigners",
        "ISCED": "ISCED Level"
    }
)
fig.update_layout(height=800, width=1200)
fig.show()


In [ ]:
fig = px.bar(
    df[df['geo'].isin(liberal)],
    x="year",
    y="Percentage",
    color="ISCED",
    facet_col="geo",
    facet_col_wrap=3,
    title="Structure of migrants’ education levels in liberal countries (stacked by year)",
    labels={
        "year": "Year",
        "Percentage": "Percentage of migrants",
        "ISCED": "Education Level"
    },
    barmode="stack"
)

fig.update_layout(height=800, width=1200)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))  
fig.show()


In [106]:
fig = px.bar(
    df[df['geo'].isin(conservs)],
    x="year",
    y="Percentage",
    color="ISCED",
    facet_col="geo",
    facet_col_wrap=3,
    title="Structure of migrants’ education levels in conservative countries (stacked by year)",
    labels={
        "year": "Year",
        "Percentage": "Percentage of migrants",
        "ISCED": "Education Level"
    },
    barmode="stack"
)

fig.update_layout(height=800, width=1200)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))  
fig.show()


Braki danych

# Population by sex, age, citizenship and labour status (1 000)

In [114]:
df = pd.read_csv('../processed_data/employment_status_foreigners.csv')

In [115]:
df.head()

,Sex,Employment status,geo,year,In thousands
0,Females,Employed persons,Austria,2014,244.8
1,Females,Employed persons,Austria,2015,253.2
2,Females,Employed persons,Austria,2016,282.5
3,Females,Employed persons,Austria,2017,293.7
4,Females,Employed persons,Austria,2018,311.2


In [116]:
pivot_df = df.pivot_table(
    index=['geo', 'year', 'Sex'],
    columns='Employment status',
    values='In thousands'
).reset_index()


In [118]:
pivot_df.head()

Employment status,geo,year,Sex,Employed persons,Persons outside the labour force,Unemployed persons
0,Austria,2014,Females,244.8,146.5,27.9
1,Austria,2014,Males,272.2,80.6,36.6
2,Austria,2014,Total,517.0,227.1,64.5
3,Austria,2015,Females,253.2,161.7,29.5
4,Austria,2015,Males,297.6,85.7,39.9


In [119]:
liberal = ["France", "Italy", "Germany", "Sweden", "Spain", "Austria"]
conservs = ["Poland", "Bulgaria", "Romania", "Hungary",  "Greece"]
selected = liberal + conservs

In [124]:
categories = ['Employed persons', 'Persons outside the labour force', 'Unemployed persons']
def plot_employment_structure(df, title):
    employment_structure = df.groupby(['geo', 'Sex', 'Employment status'], as_index=False)['In thousands'].mean()
    employment_structure['Employment status'] = pd.Categorical(employment_structure['Employment status'], categories=categories, ordered=True)
    fig = px.bar(
    employment_structure,
    x='Employment status',
    y='In thousands',
    color='Sex',
    facet_col='geo',
    facet_col_wrap=3,
    category_orders={'Employment status': categories},
    title=title,
    labels={'Employment status': 'Employment status', 'In thousands': 'Foreigners in thousands'}
)

    fig.update_layout(height=1000, width=1200, showlegend=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.show()

In [130]:
liberals_filtered = df[(df['geo'].isin(liberal)) &
                       (df['Sex'] != 'Total')]

conserve_filtered = df[(df['geo'].isin(conservs)) &
                       (df['Sex'] != 'Total')]

plot_employment_structure(liberals_filtered, "Employment structure of foreigners in selected liberal countries (2014-2024, mean)")
plot_employment_structure(conserve_filtered, "Employment structure of foreigners in selected conservative countries(2014-2024, mean)")

Dla krajów konserwatywnych w kontekście migracji bardzo mało danych - brak możliwości analizy.

In [145]:

employment_structure = liberals_filtered.groupby(['geo', 'Employment status'], as_index=False)['In thousands'].mean()
fig = px.bar(
employment_structure,
x='geo',
y='In thousands',
color='Employment status',
title="Employment structure of foreigners in selected liberal countries (2014-2024, mean)",
labels={'geo': 'Country', 'In thousands': 'Foreigners in thousands'}
)
fig.update_layout(showlegend=True)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()